# TEST POTENTIAL FUNCTIONS SFO
### The object of this section of script is to create the functions that handle the population.
### The idea is to take the population make random changes, create the fluxes matrix ( now this matrix is going to be in a subset of possible matrices in some ensemble that I do not know how to describe.) Now I renormalize the fluxes matrix.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
import time
from shapely.geometry import box
from shapely.ops import unary_union
import socket
current_dir = os.path.join(os.getcwd()) 
mother_path = os.path.abspath(os.path.join(current_dir, os.pardir))
print('mother_path:', mother_path)
sys.path.append(os.path.join(mother_path, 'PreProcessing'))
sys.path.append(os.path.join(mother_path))
from FittingProcedures import *
from plot import *
from Potential import *
from Grid import *
from PreprocessingObj import *
from ODfromfma import *
import time
plt.rcParams.update({
    "text.usetex": False,
})

if socket.gethostname()=='artemis.ist.berkeley.edu':
    TRAFFIC_DIR = '/home/alberto/LPSim/traffic_phase_transition'
else:
    TRAFFIC_DIR = os.getenv('TRAFFIC_DIR')
name = 'BOS'
grid_size = 0.02
hexagon_resolution = 8
dir_grid = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))


mother_path: /home/alberto/LPSim/traffic_phase_transition/scripts


/home/alberto/LPSim/traffic_phase_transition/scripts/PreProcessing/Potential.py:605: SyntaxWarning: invalid escape sequence '\d'
  '''


# FUNCTIONS OF INTEREST

In [2]:
## Play With Population
def ReassignPopulationRandomly(population, m0, m1, N):
    '''
        This function reassign population according to ModifyPop principles
    '''
    indices = np.where(population > 0)[0]  # Step 1

    for _ in range(N):
        index = np.random.choice(indices)  # Step 2
        rand_value = np.random.exponential(m0)  # Step 3
        exp_value = np.exp(-population[index] / m1)  # Exponential value of vector[index] / b
        if rand_value > exp_value:  # Step 4
            population[index] = 0
            index1 = np.random.choice(indices)
            population[index1] += rand_value  # Step 5
    return population

def ModifyPop(population,A,m0,m1):
    '''
        The idea is to redistribute the population in such a way that we favor the big population. Where big is given by the m1 weight rather then m0
        If m1 is bigger then m0 then the big centers have low probability of being redistributed.
        m1 big -> Toward monocentrism
        m1 -> small Toward polycentrism
    '''
    global iterations
    if isinstance(population,np.ndarray):
        pass
    else:
        population = np.array(population)
    population = ReassignPopulationRandomly(population,m0,m1,iterations)    
    return population

# Stop Playing with Population

##### PLOTTING SESSION #####
def PlotDistributionFit(ObservationVector,label,Name2Function,x_label = 'Population',y_label = 'Counts',x_scale = 'log',y_scale = 'log'):
    '''
        Input:
            ObservationVector: np.array
            label: str (powerlaw,exponential,linear)
            Name2Function: dict [Is the one coming from FittingProcedures.py]
            x_label: str (default = 'Population')
            y_label: str (default = 'Counts')
        Description:
            This function takes a distribution vector and plots it along with the exponential fit.
        
    '''
    if label == 'exponential':
        b = (bins[0] - bins[5])/np.log(n[5]/n[0])
        A = np.exp(bins[0]/b)*n[0]    
        print('A:',A)
        print('b:',b)
        initial_guess = (n[0],b)
        maxfev = 10000
    else:
        initial_guess = (1,1)
        maxfev = 10000
    if not isinstance(ObservationVector,np.ndarray):
        ObservationVector = np.array(ObservationVector)
    n,bins = np.histogram(ObservationVector,bins = 100)
    fit = Fitting(bins[:-1],n,label = label,initial_guess=initial_guess,maxfev = maxfev)
    fig,ax = plt.subplots(1,1,figsize=(10,10))
    ax.scatter(bins[:-1],n)
    plt.plot(bins[:-1],Name2Function[label](bins[:-1],fit[0][0],fit[0][1]),color = 'red')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title('{} fit'.format(label))
    ax.set_yscale(x_scale)
    ax.set_xscale(y_scale)
    plt.show()



In [3]:
# Get directories and parameters of interest
SFO_obj = GeometricalSettingsSpatialPartition(name,TRAFFIC_DIR)
bbox = SFO_obj.bounding_box
minx, miny, maxx, maxy = bbox
dir_geometries = os.path.join(TRAFFIC_DIR,'data','carto','{0}'.format(name))
grid = GetGrid(grid_size,SFO_obj.bounding_box,'EPSG:4326',dir_geometries)
df_distance,FoundDistMat = GetDirectionMatrix(dir_geometries,grid_size)
Tij = GetODGrid(dir_geometries,str(grid_size))
lattice = nx.read_graphml(os.path.join(dir_grid,"centroid_lattice.graphml"))



Initialize Grid: 0.02
ALREADY COMPUTED


# COMPUTE VECTOR FIELD

In [11]:
if True:
    VectorField = GetVectorField(Tij,df_distance)
    VectorFieldDir = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))
    SaveVectorField(VectorField,VectorFieldDir)
else:
    VectorField = GetSavedVectorFieldDF(os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size)))

# COMPUTE POTENTIAL

In [ ]:
lattice = GetPotentialLattice(lattice,VectorField)
lattice = SmoothPotential(lattice)
PotentialDataframe = ConvertLattice2PotentialDataframe(lattice)
PotentialDataframe = CompletePotentialDataFrame(VectorField,grid,PotentialDataframe)
SavePotentialDataframe(PotentialDataframe,dir_grid)
PotentialDataframe

# Correlation Mass Potential

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (12,12))
x =grid['V_out'].to_numpy()
y = grid['population'].to_numpy()
corr_coef = np.corrcoef(x, y)[0, 1]
# Fit a linear regression line
m, b = np.polyfit(x, y, 1)
ax.scatter(x,y)
ax.plot(x, m*x + b, color='red', label=f'Linear Fit (Correlation: {corr_coef:.2f})')

ax.set_xlabel('Potential')
ax.set_ylabel('Population')
plt.savefig(os.path.join(dir_grid,'PopulationPotential.png'),dpi = 200)
#ax.set_xscale('log')
#ax.set_yscale('log')

# FIT GRAVITY EXP MASS = 1


In [ ]:
import gzip

cartodir = os.path.join(TRAFFIC_DIR,'data','carto','BOS')
dirgrid = os.path.join(cartodir,'grid')
potentialdir = os.path.join(cartodir,'potential')

#with open(os.path.join(cartodir,'d0s.json'),'r') as f:
#    json.load(f)

if not os.path.exists(os.path.join(cartodir,'d0s.json')):
    d = {'d0':d0,'filteredd0':list(d0filtered.astype(float))}
    with gzip.open(os.path.join(cartodir,'d0s.json'), "wt") as json_file:
        json.dump(d, json_file)
else:
    with gzip.open(os.path.join(cartodir,'d0s.json'), "rt") as json_file:
        data = json.load(json_file)


In [ ]:
# INITIALIZING POPULATION, DISTANCE, FLUXES
distance_matrix = DistanceDf2Matrix(df_distance) # 1.2 s with 3470 grids
VgridIdx,VgridPopulation = Grid2Arrays(grid) # 0.0003 with 3470
Vnpeople,Vorigins,Vdestinations = T2Arrays(Tij) # 0.07
Vnpeople, Vorigins, Vdestinations = SubsampleFluxesByPop(Vnpeople, Vorigins, Vdestinations)

# GET d0
d0,d0s = d0PotentialFitOptimized(Vnpeople, Vorigins, Vdestinations, VgridIdx,VgridPopulation, distance_matrix)    #time spent to compute max size:  272.7686324119568
d0 = data['d0']
EstimateFluxesScaled,Fluxes,DistanceVector,ErrorEsteem,ErrorFluxes,ErrorDist,Massi,Massj = GetEstimationFluxesVector(Vnpeople, Vorigins, Vdestinations, VgridIdx, VgridPopulation, distance_matrix,d0)
# GET k
d0,k,q = GetkPotential(EstimateFluxesScaled,Fluxes,d0,potentialdir)
with open(os.path.join(potentialdir,'FitFluxesParameters.json'),'w') as f:
    json.dump({'d0':float(d0),'k':k[0],'q': k[1]},f)

# PLOT
PlotDistanceFluxes(EstimateFluxesScaled,Fluxes,DistanceVector,'No error')
PlotDistanceFluxes(ErrorEsteem,ErrorFluxes,ErrorDist,'Error')
nFlux,binsFlux = np.histogram(Fluxes,bins = 100)
nEsteem, binsEsteem = np.histogram(EstimateFluxesScaled,bins = 100)
fig,ax = plt.subplots(1,1,figsize = (10,10))
ax.scatter(binsFlux[:-1],nFlux)
ax.plot(binsEsteem[:-1],nEsteem)
ax.set_xlabel('Fluxes')
ax.set_ylabel('Distribution')
ax.set_title('Comparison distribution Measured/Gravity Fluxes')
plt.show()

In [ ]:
n,bins = np.histogram(DistanceVector,bins = 50)
AvgFluxGivenR =[Fluxes[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]]/(Massi[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]]*Massj[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]]) for i in range(len(bins)-1)]
error = [np.std(AvgFluxGivenR[i])/np.sqrt(len(AvgFluxGivenR[i])) for i in range(len(AvgFluxGivenR))]
mean = [np.median(AvgFluxGivenR[i]) for i in range(len(AvgFluxGivenR))]
fig,ax = plt.subplots(1,1,figsize = (10,10))
plt.errorbar(bins[:-1],mean,yerr = error)
plt.yscale('log')
plt.xlabel('R(km)')
plt.ylabel('Wij/(mi*mj)')
plt.title('Errorbar Plot')

# Show the plot
plt.show()

AvgMassProduct =[Massi[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]]*Massj[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]] for i in range(len(bins)-1)]
error = [np.std(AvgMassProduct[i])/np.sqrt(len(AvgMassProduct[i])) for i in range(len(AvgMassProduct))]
mean = [np.median(AvgMassProduct[i]) for i in range(len(AvgMassProduct))]
fig,ax = plt.subplots(1,1,figsize = (10,10))
plt.errorbar(bins[:-1],mean,yerr = error)
#plt.yscale('log')
plt.xlabel('R(km)')
plt.ylabel('mi*mj')
plt.title('Errorbar Plot')

# Show the plot
plt.show()

n,bins = np.histogram(DistanceVector,bins = 50)
AvgFluxGivenR =[Fluxes[np.where(DistanceVector>bins[i]) and DistanceVector < bins[i+1]] for i in range(len(bins)-1)]
error = [np.std(AvgFluxGivenR[i])/np.sqrt(len(AvgFluxGivenR[i])) for i in range(len(AvgFluxGivenR))]
mean = [np.median(AvgFluxGivenR[i]) for i in range(len(AvgFluxGivenR))]
fig,ax = plt.subplots(1,1,figsize = (10,10))
plt.errorbar(bins[:-1],mean,yerr = error)
plt.yscale('log')
plt.xlabel('R(km)')
plt.ylabel('Wij')
plt.title('Errorbar Plot')

# Show the plot
plt.show()


# FIT VESPIGNANI

In [ ]:
# INITIALIZING POPULATION, DISTANCE, FLUXES
distance_matrix = DistanceDf2Matrix(df_distance) # 1.2 s with 3470 grids
VgridIdx,VgridPopulation = Grid2Arrays(grid) # 0.0003 with 3470
Vnpeople,Vorigins,Vdestinations = T2Arrays(Tij) # 0.07
Vnpeople, Vorigins, Vdestinations = SubsampleFluxesByPop(Vnpeople, Vorigins, Vdestinations)
EstimateFluxesScaled,Fluxes,DistanceVector,ErrorEsteem,ErrorFluxes,ErrorDist,Massi,Massj = GetEstimationFluxesVector(Vnpeople, Vorigins, Vdestinations, VgridIdx, VgridPopulation, distance_matrix,d0)

# [DistanceVector,Massi,Massj], Fluxes -> 1D vectors for all couples of OD.
VespignaniVector,Fluxes = PrepareVespignani(Vnpeople, Vorigins, Vdestinations, VgridIdx, VgridPopulation, distance_matrix)

# FIT
k,error = Fitting(VespignaniVector,np.array(Fluxes),label = 'vespignani',initial_guess = [0.46,0.64,1.44,0.001] ,maxfev = 10000)

# SAVE FIT
print(os.path.join(potentialdir,'FitVespignani.json'))
with open(os.path.join(potentialdir,'FitVespignani.json'),'w') as f:
    json.dump({'logk':k[0],'alpha': k[1],'gamma': k[2],'1/d0':k[3]},f)

# TAKE THE DISTRIBUTION OF FLUXES COLLECTING BY DISTANCE BINS
n,bins = np.histogram(VespignaniVector[2],bins = 50)
EstimatedVectorFluxesVespignani = multilinear4variables(VespignaniVector,k[0],k[1],k[2],k[3])
AvgFluxGivenR = [EstimatedVectorFluxesVespignani[np.where(((VespignaniVector[2]>bins[i]) & (VespignaniVector[2] < bins[i+1])))]/(VespignaniVector[0][np.where(((VespignaniVector[2]>bins[i]) & (VespignaniVector[2] < bins[i+1])))]**k[0]*VespignaniVector[1][np.where(((VespignaniVector[2]>bins[i]) & (VespignaniVector[2] < bins[i+1])))])**k[1] for i in range(len(bins)-2)]
error = [np.std(AvgFluxGivenR[i])/np.sqrt(len(AvgFluxGivenR[i])) for i in range(len(AvgFluxGivenR))]
mean = [np.median(AvgFluxGivenR[i]) for i in range(len(AvgFluxGivenR))]

# PLOTTING
fig,ax = plt.subplots(1,1,figsize = (10,10))
plt.errorbar(bins[:-2],mean,yerr = error, fmt='o', capsize=5, color='red')
plt.yscale('log')
plt.xlabel('R(km)')
plt.ylabel('$W_{ij}/(m_i^{{\\alpha}} m_j^{{\\gamma}})$')
plt.savefig(os.path.join(potentialdir,'PlotFitVespignani.png'),dpi = 200)
plt.show()


# REASSIGN MASS -> POTENTIAL

In [114]:
## Play With Population
def ReassignPopulationRandomly(population, m0, m1, N):
    '''
        This function reassign population according to ModifyPop principles
    '''
    indices = np.where(population > 0)[0]  # Step 1

    for _ in range(N):
        index = np.random.choice(indices)  # Step 2
        rand_value = np.random.exponential(m0)  # Step 3
        exp_value = np.exp(-population[index] / m1)  # Exponential value of vector[index] / b
        if rand_value > exp_value:  # Step 4
            population[index] = 0
            index1 = np.random.choice(indices)
            population[index1] += rand_value  # Step 5
    return population

def ModifyPop(population,A,m0,m1):
    '''
        The idea is to redistribute the population in such a way that we favor the big population. Where big is given by the m1 weight rather then m0
        If m1 is bigger then m0 then the big centers have low probability of being redistributed.
        m1 big -> Toward monocentrism
        m1 -> small Toward polycentrism
    '''
    global iterations
    if isinstance(population,np.ndarray):
        pass
    else:
        population = np.array(population)
    population = ReassignPopulationRandomly(population,m0,m1,iterations)    
    return population

# Stop Playing with Population
def PlotLorenzCurve(cumulative,Fstar,label2color,result_indices,label,save_dir,shift = 500):
    line1, = plt.plot(np.arange(len(cumulative)),cumulative,c='black',label=label)
    # Plot the straight line to F*
    line2, = plt.plot([Fstar, len(cumulative)], [0, cumulative[-1]], color='red',label = label + ' angle')
    plt.plot(Fstar,cumulative[Fstar],'ro',label=label + ' F*')
    plt.axvline(x=Fstar, color=label2color[label], linestyle='--')
    if result_indices is not None:
        fraction = len(result_indices) / len(cumulative)
        plt.text(Fstar + shift, 0, f'I* = {fraction:.2f}', ha='right', va='bottom', color='black')
        plt.text(Fstar + 2*shift , 0, f'Centers', ha='right', va='bottom', color='green')
        plt.text(Fstar - 1.5*shift , 0, f'No Centers', ha='right', va='bottom', color='yellow')        
        plt.axhline(y=0, xmin= Fstar / len(cumulative), xmax=1, color='green', linestyle='--')
        plt.axhline(y=0, xmin=0, xmax=(1 - fraction), color='yellow', linestyle='--')
        print(len(cumulative)*(1 - Fstar / len(cumulative)))
    ax.set_ylim(0)
    plt.title('Lorenz Curve Potential')
    plt.xlabel('Index sorted grid')
    plt.ylabel('Cumulative Potential')
    plt.savefig(os.path.join(save_dir,'LoufCurve.png'),dpi = 200)
    return line1,line2

def LorenzCenters(potential):
    '''
        Input:
            Potential from grid.
        This function computes the indices of the centers in the linearized grid.
        We are using here the index column and not the double index.
    '''
    # Set potential having just positive values
    potential = potential - min(potential)
    # Step 1: Sort the potential and compute the sorting map
    sorted_indices = np.argsort(potential)
    # Step 2: Compute the cumulative distribution
    sorted_potential = potential[sorted_indices] 
    print('sorted potential: ',sorted_potential)
    cumulative = np.cumsum(sorted_potential)
    # Step 3: Determine the angle and delta index
    angle = cumulative[-1] - cumulative[-2]
    Fstar = int(len(cumulative) + 1 - cumulative[-1]/angle)
#    plt.show()
    # Step 4: Retrieve the indices based on the delta index and mapping
    result_indices = [sorted_indices[-i] for i in range(len(cumulative) - Fstar)]
    fraction = len(result_indices)/len(potential)
    print('The angle: ',angle)
    print('Fstar: ',Fstar)
    print('Fraction of indices considered centers: ',len(result_indices)/len(potential))
    return result_indices,angle,cumulative,Fstar,fraction




In [ ]:
label2color = {'data':'red'}
result_indices,angle,cumulative,Fstar,fraction = LorenzCenters(grid['V_out'].to_numpy())
PlotLorenzCurve(cumulative,Fstar,label2color,result_indices,'data',dir_grid)

# POPULATION FIT

In [ ]:
# TODO: Suppose you have a population distribution that is an exponential with fitting parameter m0. I want to redistribute the mass in such a way
# that I take favor the big population, then I fix 

import matplotlib.patches as patches
n,bins = np.histogram(grid['population'].to_numpy(),bins = 40)
# FILTER MASS
def FilterPercentageCumulative(perc,n,bins):
    total_sum = np.sum(n) 
    cumulative_sum = np.cumsum(n)
    index_95_percent = np.argmax(cumulative_sum >=  perc* total_sum)
    # Select the elements that contain 95% of the sum
    n = n[:index_95_percent + 1]
    bins = bins[:index_95_percent + 1]
    return n,bins
start_bin = 1
end_bin = 0

if len(n) == len(bins):
    m0 = (bins[-end_bin] - bins[start_bin])/np.log(n[start_bin]/n[-end_bin])
    Aend = n[-end_bin]*np.exp(bins[-end_bin]/m0)
    Astart = n[start_bin-1]*np.exp(bins[start_bin]/m0)
    ResultFit = Fitting(bins[start_bin:-end_bin],n[start_bin:-end_bin],label = 'powerlaw',initial_guess = (11,-1.83),maxfev = 10000)
    coefficients = np.polyfit(bins[start_bin:-end_bin],np.log(n[start_bin:-end_bin]), 1)
    poly_function = np.poly1d(coefficients)
    y_fit = poly_function(bins[start_bin:-end_bin])
    plt.scatter(bins[start_bin:-end_bin],n[start_bin:-end_bin])
    plt.plot(bins[start_bin:-end_bin],y_fit)
#    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    print(coefficients)

else:
    ResultFit = Fitting(bins[start_bin:],n[start_bin-1:],label = 'powerlaw',initial_guess = (11,-1.83),maxfev = 10000)
    plt.scatter(bins[start_bin:],n[start_bin-1:])
    plt.plot(bins[start_bin:],bins[start_bin:]**ResultFit[0][1]*ResultFit[0][0])
    plt.xlabel('m (Population)')
    plt.ylabel('Distribution')
    plt.xscale('log')
    plt.yscale('log')
    plt.title('Power law fit')# = {}'.format(round(ResultFit[0][1],2)))
    plt.savefig(os.path.join(dir_grid,'PopulationDistribution.png'),dpi = 200)
    plt.show()


#A, m0 = ResultFit[0],ResultFit[1]
#iterations = len(grid['population'].to_numpy())
#amplifications = np.arange(0.1,10,100)
#for amp in amplifications:
#    ModifyPop(grid['population'],A,m0,amp*m0)


In [ ]:
print('Total population: ',np.sum(grid['population']),' Moving population: ',np.sum(Tij['number_people']),'Fraction people moving from population: ',np.sum(Tij['number_people'])/np.sum(grid['population']))

Total population:  4546481.94410453  Moving population:  681885 Fraction people moving from population:  0.14998080018424076


# PLOT POTENTIAL AND VECTOR FIELDS 

### Vector field with backgroudn heatmap potential and population

In [ ]:
PlotVFPotMass(grid,SFO_obj,PotentialDataframe,VectorField,label_potential = 'V_out',label_fluxes = 'Ti')
PlotVFPotMass(grid,SFO_obj,PotentialDataframe,VectorField,label_potential = 'population',label_fluxes = 'Ti')


### Contour Plot and 3D Potential

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming you have a GeoDataFrame named 'grid' with a 'geometry' column containing polygons and a 'potential' column
grid['potential'] = PotentialDataframe['V_out']
# Create a contour plot
fig, ax = plt.subplots(figsize=(20, 20))
SFO_obj.gdf_polygons.plot(ax=ax, color='white', edgecolor='black',alpha = 0.2)
grid.plot(ax=ax, edgecolor='black', facecolor='none',alpha = 0.2)
contour = ax.tricontour(grid['geometry'].apply(lambda geom: geom.centroid.x), 
                         grid['geometry'].apply(lambda geom: geom.centroid.y), 
                         grid['potential'], alpha=1, cmap='inferno')
#cbar = plt.colorbar(contour)
#cbar.set_label('Potential')
ax.set_title('Curve Level of Potential')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.savefig(os.path.join(dir_grid,'CountorPlot.png'),dpi = 200)
plt.show()


x = np.linspace(min(grid.centroidx), max(grid.centroidx), len(np.unique(grid['j'])))
y = np.linspace(min(grid.centroidy), max(grid.centroidy), len(np.unique(grid['i'])))
X, Y = np.meshgrid(x, y)
Z = grid['potential'].values.reshape((len(y),len(x)))# Check the lengths of x, y, and the potential values array

print("Length of x:", len(x))
print("Length of y:", len(y))
print('Shape X,Y: ',np.shape(X))
print("Shape of Z (potential values):", np.shape(Z))

# Plot the 3D surface
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none')
#polygon = SFO_obj.gdf_polygons.to_crs(grid.crs)  # Ensure polygon has the same CRS as the grid
#polygon_patch = polygon.boundary.plot(ax=ax, color='black', alpha=0.5)
#polygon_patch.set_zorder(10)  # Ensure polygon is plotted above the surface

cbar = fig.colorbar(surf, ax=ax)
cbar.set_label('Potential Height')

ax.set_title('3D Surface Plot of Potential')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Potential')
plt.savefig(os.path.join(dir_grid,'Potential3D.png'),dpi = 200)
plt.show()


### ROTOR Heatmap and Distribution

In [ ]:
grid['rotor'] = PotentialDataframe['rotor_z_out']
# Create a contour plot
fig, ax = plt.subplots(figsize=(20, 20))
SFO_obj.gdf_polygons.plot(ax=ax, color='white', edgecolor='black',alpha = 0.2)
grid_plot = grid.plot(ax=ax, column = 'rotor', cmap = 'viridis',edgecolor='black',alpha = 0.8)
grid_cbar = plt.colorbar(grid_plot.get_children()[1], ax=ax)
grid_cbar.set_label('{}'.format('rotor'), rotation=270, labelpad=15)
ax.set_title('Rotor')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.savefig(os.path.join(dir_grid,'RotorPlot.png'),dpi = 200)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
twin = ax.twinx()
ax.hist(grid['rotor'],bins = 50, color = 'blue',label = 'Rotor')
ax.set_title('Rotor Distribution')
ax.set_xlabel('Rotor')
ax.set_ylabel('Count')
plt.savefig(os.path.join(dir_grid,'RotorDistr.png'),dpi = 200)
plt.show()


### Population for Grid and Hexagons

In [ ]:
# POPULATION GRID
fig,ax = plt.subplots(1,1,figsize = (12,12))
SFO_obj.gdf_polygons.plot(ax=ax, color='white', edgecolor='black')
grid_plot = grid.plot(column='population', cmap='viridis', edgecolor='black', alpha=0.3, legend=True, ax=ax)
ax.set_title('Population Boston')

plt.savefig(os.path.join(dir_grid,'populationBoston.png'),dpi = 200)


In [ ]:
# POPULATION HEXAGONS
fig,ax = plt.subplots(1,1,figsize = (12,12))
hexagons = gpd.read_file('/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/hexagon/8/hexagon.geojson')
SFO_obj.gdf_polygons.plot(ax=ax, color='white', edgecolor='black')
grid_plot = hexagons.plot(column='population', cmap='viridis', edgecolor='black', alpha=0.3, legend=True, ax=ax)
ax.set_title('Population Boston')
plt.savefig(os.path.join(dir_grid,'populationBostonHexagons.png'),dpi = 200)


# MODIFY POPULATION 

In [ ]:

def ComputeVectorFieldModified(Tij,df_distance):
    Tij['vector_flux'] = df_distance['dir_vector'].apply(lambda x: parse_dir_vector(x) ) * Tij['number_people']

    # Create VectorField DataFrame
    VectorField = pd.DataFrame(index=Tij['(i,j)D'].unique(), columns=['(i,j)', 'Ti', 'Tj'])
    Tj_values = Tij.groupby('(i,j)D')['vector_flux'].sum()
    VectorField['Tj'] = Tj_values

    # Calculate 'Ti' values
    Ti_values = Tij.groupby('(i,j)O')['vector_flux'].sum()
    VectorField['Ti'] = Ti_values
    VectorField['index'] = VectorField.index
    VectorField['(i,j)'] = VectorField['index']
    VectorField['index'] = VectorField.index
    VectorField.reset_index(inplace=True)
    return VectorField


## GET VF & POT

In [ ]:
ComputeInitPotential = False
ComputeInitVF = False
if ComputeInitVF:
    VectorField = GetVectorField(Tij,df_distance)
    VectorFieldDir = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))
    SaveVectorField(VectorField,VectorFieldDir)

if ComputeInitPotential:
    lattice = GetPotentialLattice(lattice,VectorField)
    lattice = SmoothPotential(lattice)
    PotentialDataframe = ConvertLattice2PotentialDataframe(lattice)
    PotentialDataframe = CompletePotentialDataFrame(VectorField,grid,PotentialDataframe)
    SavePotentialDataframe(PotentialDataframe,dir_grid)


## MODIFY

In [339]:
## TODO: Modify Population -> Compute Fluxes -> Get Vector Field -> Compute Modified Potential -> Compute Lorenz -> Compute OD

with open('/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/potential/FitFluxesParameters.json','r')as f:
    fitGLM = json.load(f)
p0 = fraction

Population = ModifyPop(Population)
Tij = ComputeFluxesModifiedPopulation(Population) 
VectorField = ComputeVectorFieldModified(Tij,df_distance)
Potential = ComputePotentialModified(VectorField)

p = ComputeLorenzCurve(Potential)
OD_DemandFromFluxes()

,origin,destination,number_people,"(i,j)O","(i,j)D",vector_flux
0,0,0,0,"(0, 0)","(0, 0)","[0, 0]"
1,0,1,0,"(0, 0)","(0, 1)","[0.0, 0.0]"
2,0,2,0,"(0, 0)","(0, 2)","[0.0, 0.0]"
3,0,3,0,"(0, 0)","(0, 3)","[0.0, 0.0]"
4,0,4,0,"(0, 0)","(0, 4)","[0.0, 0.0]"
...,...,...,...,...,...,...
12089524,3476,3472,0,"(60, 56)","(60, 52)","[0.0, -0.0]"
12089525,3476,3473,0,"(60, 56)","(60, 53)","[0.0, -0.0]"
12089526,3476,3474,0,"(60, 56)","(60, 54)","[0.0, -0.0]"
12089527,3476,3475,0,"(60, 56)","(60, 55)","[0.0, -0.0]"


In [345]:
Tij

,origin,destination,number_people,"(i,j)O","(i,j)D",vector_flux
0,0,0,0,"(0, 0)","(0, 0)","[0, 0]"
1,0,1,0,"(0, 0)","(0, 1)","[0.0, 0.0]"
2,0,2,0,"(0, 0)","(0, 2)","[0.0, 0.0]"
3,0,3,0,"(0, 0)","(0, 3)","[0.0, 0.0]"
4,0,4,0,"(0, 0)","(0, 4)","[0.0, 0.0]"
...,...,...,...,...,...,...
12089524,3476,3472,0,"(60, 56)","(60, 52)","[0.0, -0.0]"
12089525,3476,3473,0,"(60, 56)","(60, 53)","[0.0, -0.0]"
12089526,3476,3474,0,"(60, 56)","(60, 54)","[0.0, -0.0]"
12089527,3476,3475,0,"(60, 56)","(60, 55)","[0.0, -0.0]"


# ASSIGN OD FROM MODIFIED FLUXES

In [346]:
with open('/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/grid/0.02/origindest2grid.json','r'):

## PLOT

In [ ]:
# VECTOR FIELD AND POTENTIAL
PlotVFPotMass(grid,SFO_obj,PotentialDataframe,VectorField,label_potential = 'V_out',label_fluxes = 'Ti')
PlotVFPotMass(grid,SFO_obj,PotentialDataframe,VectorField,label_potential = 'population',label_fluxes = 'Ti')


In [ ]:
# LORENZ CURVE
result_indices,angle,cumulative,Fstar = LorenzCenters(grid['V_out'].to_numpy())
PlotLorenzCurve(cumulative,Fstar,label2color,result_indices,'data',dir_grid)